#### **Bu notebook'un amacı:**
1. Domain knowledge ile meaningful features yaratmak
2. Her feature'ın "neden yarattık?" sorusunu cevaplamak
3. Modüler, tekrar kullanılabilir kod yazmak
4. 04_Model_Comparison'da feature selection için hazırlık

#### **Feature Engineering Felsefesi:**
- "More is NOT better" → Quality > Quantity
- Her feature bir hipotez test eder
- Feature selection 04'te yapacağız

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [2]:
# Seed for reproducibility
np.random.seed(42)

In [3]:
df = pd.read_csv('../data/marketing_analytics_cleaned.csv')

print("📊 Dataset Shape:", df.shape)
print("\n✅ Target Variable Distribution:")
print(df['Conversion'].value_counts())
print(f"\nConversion Rate: {df['Conversion'].mean() * 100:.2f}%")


📊 Dataset Shape: (48000, 20)

✅ Target Variable Distribution:
Conversion
0    47393
1      607
Name: count, dtype: int64

Conversion Rate: 1.26%


#### **FEATURE ENGINEERING - ROI & COST METRICS**
**NEDEN BU FEATURE'LAR?**

Marketing'de en önemli metrik: ROI (Return on Investment)
"Bu müşteriye harcadığımız para ne kadar verimli?"

_HYPOTHESIS_:
- Düşük CPA (Cost Per Acquisition) olan channel'lar daha iyi
- Yüksek ROI proxy'si olan müşteriler convert olma eğiliminde


In [4]:
print("\n" + "="*70)
print("FEATURE GROUP 1: ROI & COST METRICS")
print("="*70)

# Feature 1: Cost Per Acquisition (CPA) Proxy
# Formula: AdSpend / (Conversion + 1)
# Neden +1? Zero division protection (henüz convert olmamış müşteriler için)
df['CPA_Proxy'] = df['AdSpend'] / (df['Conversion'] + 1)

print("\n✅ CPA_Proxy created")
print(f"   Mean: ${df['CPA_Proxy'].mean():.2f}")
print(f"   Interpretation: Ortalama müşteri başına maliyet proxy")


FEATURE GROUP 1: ROI & COST METRICS

✅ CPA_Proxy created
   Mean: $2189.35
   Interpretation: Ortalama müşteri başına maliyet proxy


In [5]:
# Feature 2: ROI Proxy
# Formula: (ConversionRate * Income) / AdSpend
# Mantık: Potansiyel değer / Harcanan para
# Yüksek income + yüksek conversion rate + düşük AdSpend = İyi ROI
df['ROI_Proxy'] = (df['ConversionRate'] * df['Income']) / (df['AdSpend'] + 1)

print("\n✅ ROI_Proxy created")
print(f"   Mean: {df['ROI_Proxy'].mean():.2f}")
print(f"   Interpretation: Potansiyel customer value / Marketing cost")


✅ ROI_Proxy created
   Mean: 1.11
   Interpretation: Potansiyel customer value / Marketing cost


In [7]:
# Feature 3: Spend Efficiency
# Formula: ClickThroughRate / AdSpend
# Mantık: Harcanan para başına tıklama verimliliği
df['Spend_Efficiency'] = df['ClickThroughRate'] / (df['AdSpend'] + 1)

print("\n✅ Spend_Efficiency created")
print(f"   Mean: {df['Spend_Efficiency'].mean():.6f}")
print(f"   Interpretation: AdSpend verimliliği (CTR per dollar)")


✅ Spend_Efficiency created
   Mean: 0.000073
   Interpretation: AdSpend verimliliği (CTR per dollar)


#### **FEATURE ENGINEERING - ENGAGEMENT METRICS**

**NEDEN BU FEATURE'LAR?**

User engagement = Conversion'ın en güçlü göstergelerinden biri
"Müşteri ne kadar engaged?"

**HYPOTHESIS:**
- Sitede daha fazla zaman geçiren → Daha fazla ilgili → Conversion ↑
- Daha fazla sayfa gezen → Daha fazla araştıran → Conversion ↑
- Email açan ama tıklamayan → İlgili ama kararsız

In [8]:
print("\n" + "="*70)
print("FEATURE GROUP 2: ENGAGEMENT METRICS")
print("="*70)


FEATURE GROUP 2: ENGAGEMENT METRICS


In [9]:

# Feature 4: Site Engagement Score
# Formula: TimeOnSite * PagesPerVisit
# Mantık: Derinlemesine engagement (hem süre hem depth)
df['Site_Engagement'] = df['TimeOnSite'] * df['PagesPerVisit'].fillna(1)

print("\n✅ Site_Engagement created")
print(f"   Mean: {df['Site_Engagement'].mean():.2f}")
print(f"   Interpretation: Engagement depth (time × pages)")


✅ Site_Engagement created
   Mean: 13.77
   Interpretation: Engagement depth (time × pages)


In [10]:
# Feature 5: Average Time Per Page
# Formula: TimeOnSite / PagesPerVisit
# Mantık: Her sayfada ne kadar vakit geçiriyor? (Bounce rate proxy)
df['Avg_Time_Per_Page'] = df['TimeOnSite'] / (df['PagesPerVisit'].fillna(1))

print("\n✅ Avg_Time_Per_Page created")
print(f"   Mean: {df['Avg_Time_Per_Page'].mean():.2f}")
print(f"   Interpretation: Time per page (bounce rate inverse)")



✅ Avg_Time_Per_Page created
   Mean: 1.13
   Interpretation: Time per page (bounce rate inverse)


In [11]:
# Feature 6: CTR to Conversion Ratio
# Formula: ConversionRate / ClickThroughRate
# Mantık: Tıklama → Conversion dönüşüm verimliliği
df['CTR_to_Conversion'] = df['ConversionRate'] / (df['ClickThroughRate'] + 0.0001)

print("\n✅ CTR_to_Conversion created")
print(f"   Mean: {df['CTR_to_Conversion'].mean():.2f}")
print(f"   Interpretation: Click → Conversion efficiency")



✅ CTR_to_Conversion created
   Mean: 16.71
   Interpretation: Click → Conversion efficiency


In [12]:
# Feature 7: Email Engagement
# Formula: EmailClicks / (EmailOpens + 1)
# Mantık: Email açanların ne kadarı tıklıyor?
df['Email_Click_Rate'] = df['EmailClicks'] / (df['EmailOpens'] + 1)

print("\n✅ Email_Click_Rate created")
print(f"   Mean: {df['Email_Click_Rate'].mean():.3f}")
print(f"   Interpretation: Email engagement quality")


✅ Email_Click_Rate created
   Mean: 0.056
   Interpretation: Email engagement quality


In [13]:
# Feature 8: Social Virality
# Formula: SocialShares / (WebsiteVisits + 1)
# Mantık: Ziyaret başına paylaşım eğilimi
df['Social_Virality'] = df['SocialShares'] / (df['WebsiteVisits'] + 1)

print("\n✅ Social_Virality created")
print(f"   Mean: {df['Social_Virality'].mean():.3f}")
print(f"   Interpretation: Share propensity per visit")


✅ Social_Virality created
   Mean: 0.857
   Interpretation: Share propensity per visit


#### **FEATURE ENGINEERING - CUSTOMER SEGMENTATION**
**NEDEN BU FEATURE'LAR?**

Farklı müşteri segmentleri farklı davranır
"Bu müşteri hangi profile yakın?"

**HYPOTHESIS:**
- Genç vs yaşlı müşteriler farklı channel'larda aktif
- Yüksek income müşteriler farklı campaign'lere tepki verir
- Loyalty tier'ı yüksek müşteriler daha fazla convert olur

In [14]:
print("\n" + "="*70)
print("FEATURE GROUP 3: CUSTOMER SEGMENTATION")
print("="*70)


FEATURE GROUP 3: CUSTOMER SEGMENTATION


In [15]:
# Feature 9: Age Group (Binning)
# Kategoriler: Young Adult, Adult, Middle Age, Senior
df['Age_Group'] = pd.cut(df['Age'],
                          bins=[0, 25, 35, 50, 100],
                          labels=['YoungAdult', 'Adult', 'MiddleAge', 'Senior'])

print("\n✅ Age_Group created")
print(df['Age_Group'].value_counts())


✅ Age_Group created
Age_Group
Adult         19258
MiddleAge     16183
Senior         8376
YoungAdult     4183
Name: count, dtype: int64


In [16]:
# Feature 10: Income Tier (Quantile-based)
# Kategoriler: Low, Medium, High, VeryHigh
df['Income_Tier'] = pd.qcut(df['Income'],
                              q=4,
                              labels=['Low', 'Medium', 'High', 'VeryHigh'],
                              duplicates='drop')

print("\n✅ Income_Tier created")
print(df['Income_Tier'].value_counts())


✅ Income_Tier created
Income_Tier
Low         12001
High        12000
VeryHigh    12000
Medium      11999
Name: count, dtype: int64


In [17]:
# Feature 11: Loyalty Tier (Quantile-based)
# Kategoriler: Bronze, Silver, Gold
df['Loyalty_Tier'] = pd.qcut(df['LoyaltyPoints'],
                               q=3,
                               labels=['Bronze', 'Silver', 'Gold'],
                               duplicates='drop')

print("\n✅ Loyalty_Tier created")
print(df['Loyalty_Tier'].value_counts())


✅ Loyalty_Tier created
Loyalty_Tier
Bronze    16000
Silver    16000
Gold      16000
Name: count, dtype: int64


In [18]:
# Feature 12: Customer Value Score
# Formula: PreviousPurchases * LoyaltyPoints * Income (normalized)
# Mantık: Customer Lifetime Value (CLV) proxy
df['Customer_Value_Score'] = (
    df['PreviousPurchases'] *
    df['LoyaltyPoints'] *
    (df['Income'] / df['Income'].max())  # Normalize income
)

print("\n✅ Customer_Value_Score created")
print(f"   Mean: {df['Customer_Value_Score'].mean():.2f}")
print(f"   Interpretation: CLV proxy (purchase history × loyalty × income)")


✅ Customer_Value_Score created
   Mean: 1466.69
   Interpretation: CLV proxy (purchase history × loyalty × income)


#### **FEATURE ENGINEERING - INTERACTION FEATURES**

**NEDEN BU FEATURE'LAR?**

Interaction features = Non-linear relationships yakalama
"İki feature birlikte nasıl etkiliyor?"

**HYPOTHESIS:**
- Yüksek AdSpend × Yüksek CTR = Güçlü sinerji
- Yüksek Income × Yüksek Loyalty = Premium customer

In [19]:
print("\n" + "="*70)
print("FEATURE GROUP 4: INTERACTION FEATURES")
print("="*70)


FEATURE GROUP 4: INTERACTION FEATURES


In [20]:
# Feature 13: AdSpend × CTR (Marketing Effectiveness)
df['AdSpend_x_CTR'] = df['AdSpend'] * df['ClickThroughRate']

print("\n✅ AdSpend_x_CTR created")
print(f"   Mean: {df['AdSpend_x_CTR'].mean():.2f}")



✅ AdSpend_x_CTR created
   Mean: 173.70


In [21]:
# Feature 14: Income × LoyaltyPoints (Premium Customer Indicator)
df['Income_x_Loyalty'] = (df['Income'] / 10000) * (df['LoyaltyPoints'] / 1000)

print("\n✅ Income_x_Loyalty created")
print(f"   Mean: {df['Income_x_Loyalty'].mean():.2f}")


✅ Income_x_Loyalty created
   Mean: 5.03


In [22]:
# Feature 15: Age × PreviousPurchases (Experienced Customer)
df['Age_x_Purchases'] = df['Age'] * df['PreviousPurchases']

print("\n✅ Age_x_Purchases created")
print(f"   Mean: {df['Age_x_Purchases'].mean():.2f}")


✅ Age_x_Purchases created
   Mean: 57.70
